In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
import matplotlib.pyplot as plt
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
import os
from scipy.stats import norm


In [ ]:
 data_path = os.path.join("..", "data")

In [ ]:
X = np.load(data_path + '/initial_data/function_2/initial_inputs.npy')
y = np.load(data_path + '/initial_data/function_2/initial_outputs.npy')
print(f"X is:{X}")
print(f"Y is :{y}")

In [ ]:
# add the week 1 input and result to the array
X = np.append(X, [[0.795918,0.959184 ]], axis=0)
y = np.append(y, [0.078279871], axis=0)

# add the week 2 input and result to the array
X = np.append(X, [[0.714286, 0.938776]], axis=0)
y = np.append(y, [0.5239853412950183], axis=0)

# add the week 3 input and result to the array
X = np.append(X, [[0.656566, 0.939394]], axis=0)
y = np.append(y, [0.414404463864519], axis=0)

# add the week 4 input and result to the array
X = np.append(X, [[0.729253, 0.058662]], axis=0)
y = np.append(y, [0.5073132554292137], axis=0)

# add the week 5 input and result to the array
X = np.append(X, [[0.999980, 0.502674]], axis=0)
y = np.append(y, [-0.007820208557515348], axis=0)

# add the week 6 input and result to the array
X = np.append(X, [[0.727273, 0.848485]], axis=0)
y = np.append(y, [0.5692408682454391], axis=0)

# add the week 7 input and result to the array
X = np.append(X, [[0.767677, 0.171717]], axis=0)
y = np.append(y, [0.19283512655684448], axis=0)

# add the week 8 input and result to the array
X = np.append(X, [[0.636364, 0.040404]], axis=0)
y = np.append(y, [0.14715688566886923], axis=0)

# add the week 9 input and result to the array
X = np.append(X, [[0.696970, 0.858586]], axis=0)
y = np.append(y, [0.6356443554454597], axis=0)

# add the week 10 input and result to the array
X = np.append(X, [[0.707071, 0.888889]], axis=0)
y = np.append(y, [0.7207495222038984], axis=0)

# add the week 11 input and result to the array
X = np.append(X, [[0.707071, 0.888889]], axis=0)
y = np.append(y, [0.590410153454048], axis=0)

# add the week 12 input and result to the array
X = np.append(X, [[0.707071, 0.878788]], axis=0)
y = np.append(y, [0.5971877082244439], axis=0)

print(f"X is:{X}")
print(f"Y is :{y}")

In [ ]:
print(f"max y is :{np.max(y)}")

In [ ]:
# Define GP model
kernel = Matern(nu=2.5, length_scale_bounds=(1e-2, 1e2))
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-10, normalize_y=True)
gp.fit(X, y)

In [ ]:
# UCB acquisition function
# week 6 changed kappa to 4.0 from 2.0
# week 8 chnaged kappa to 3.0 from 4.0 for more exploitation
def ucb(x, gp, kappa=3.0):
    x = x.reshape(1, -1)
    mu, sigma = gp.predict(x, return_std=True)
    return mu + kappa * sigma

In [ ]:
# Expected Improvement (EI) acquisition function
# Week 11 change xi to 0.001 from 0.01 for more exploitation    
# Week 12 change xi to 0.01 
def expected_improvement(x, gp, y_best, xi=0.01):
    x = x.reshape(1, -1)
    mu, sigma = gp.predict(x, return_std=True)
    mu = mu[0]
    sigma = sigma[0]
    
    if sigma == 0:
        return 0.0
    
    Z = (mu - y_best - xi) / sigma
    ei = (mu - y_best - xi) * norm.cdf(Z) + sigma * norm.pdf(Z)
    return ei

In [ ]:
grid_size = 100
x1 = np.linspace(0, 1, grid_size)
x2 = np.linspace(0, 1, grid_size)
X_grid = np.array([[a, b] for a in x1 for b in x2])

In [ ]:
ucb_values = np.array([ucb(x, gp) for x in X_grid])


In [ ]:
# Compute EI values
y_best = np.max(y)
ei_values = np.array([expected_improvement(x, gp, y_best) for x in X_grid])

In [ ]:
best_idx = np.argmax(ucb_values)
next_best_point = X_grid[best_idx]
pred_mean, pred_std = gp.predict(next_best_point.reshape(1, -1), return_std=True)
print("Predicted maximum value (mean):", pred_mean[0])
print("Next best point to evaluate (UCB):", np.round(next_best_point,6))
print("Next best point to evaluate (UCB) without rounding:", next_best_point)
print("Predicted maximum value (mean):", pred_mean[0])

In [ ]:
# Select next best point
best_idx = np.argmax(ei_values)
next_best_point = X_grid[best_idx]
pred_mean, pred_std = gp.predict(next_best_point.reshape(1, -1), return_std=True)

print("Next best point to evaluate (EI):", np.round(next_best_point, 6))
print("Next best point to evaluate (EI) without rounding:", next_best_point)
print("Predicted mean at next point:", pred_mean[0])
print("Expected Improvement value:", ei_values[best_idx])

In [ ]:
# Scale inputs
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train SVM surrogate
svm_surrogate = SVR(kernel='rbf', C=10.0, epsilon=0.1)
svm_surrogate.fit(X_scaled, y)

# Predict on new candidates
X_new = np.random.rand(10000000, 2)
X_new_scaled = scaler.transform(X_new)
y_pred = svm_surrogate.predict(X_new_scaled)

# Maximization: find best candidate
best_idx = np.argmax(y_pred)
best_candidate = X_new[best_idx]
best_predicted_value = y_pred[best_idx]


print("Best candidate input:", best_candidate)
print("Best candidate input rounded:", np.round(best_candidate, 6))
print("Predicted maximum value:", best_predicted_value)